In [1]:
# load packages
import impectPy
import numpy as np
import pandas as pd

In [ ]:
# Set login credentials
username = "yourUsername"
password = "yourPassword"

# create Impect instance and login
api = impectPy.Impect()
api.login(username=username, password=password)

In [ ]:
# set iterationId
iteration = 1385

# fetch matches for iteration
matches = api.getMatches(iteration=iteration)

# show matches dataframe
matches.head()

In [ ]:
# fetch prediction model coefficients
coefficients = api.getSquadCoefficients(iteration=iteration)

# show coefficients
coefficients.head()

In [ ]:
# prepare columns for merging
matches["date"] = pd.to_datetime(matches["scheduledDate"]).dt.tz_localize(None).dt.normalize()
matches["homeSquadId"] = matches["homeSquadId"].astype("int64")
matches["awaySquadId"] = matches["awaySquadId"].astype("int64")
coefficients["date"] = pd.to_datetime(coefficients["date"]).dt.normalize()
coefficients["squadId"] = coefficients["squadId"].astype("int64")

# sort by date
matches = matches.sort_values("date")
coefficients = coefficients.sort_values("date")

# merge competition-specific coefficients using the most recent date
matches = pd.merge_asof(
    matches,
    coefficients[
        ["date", "interceptCoefficient", "homeCoefficient", "competitionCoefficient"]
    ].drop_duplicates("date"),
    on="date",
    direction="backward"
)

# merge squad-specific coefficients using the most recent date
def get_squad_coeffs(row, coeff_df, squad_id_col) -> pd.Series:
    squad_id = row[squad_id_col]
    match_date = row["date"]
    squad_coeffs = coeff_df[(coeff_df["squadId"] == squad_id) & (coeff_df["date"] <= match_date)]
    if len(squad_coeffs) > 0:
        latest = squad_coeffs.sort_values("date").iloc[-1]
        return pd.Series({"attack": latest["attackCoefficient"], "defense": latest["defenseCoefficient"]})
    return pd.Series({"attack": None, "defense": None})

# merge homeSquad coefficients
home_coeffs = matches.apply(lambda row: get_squad_coeffs(row, coefficients, "homeSquadId"), axis=1)
matches["attackCoefficientHome"] = home_coeffs["attack"]
matches["defenseCoefficientHome"] = home_coeffs["defense"]

# merge awaySquad coefficients
away_coeffs = matches.apply(lambda row: get_squad_coeffs(row, coefficients, "awaySquadId"), axis=1)
matches["attackCoefficientAway"] = away_coeffs["attack"]
matches["defenseCoefficientAway"] = away_coeffs["defense"]

# show new matches dataframe
matches.head()

In [ ]:
# compute predictions
matches["predHome"] = np.exp(
    matches["interceptCoefficient"] +
    matches["homeCoefficient"] +
    matches["competitionCoefficient"] +
    matches["attackCoefficientHome"] +
    matches["defenseCoefficientAway"]
)
matches["predAway"] = np.exp(
    matches["interceptCoefficient"] +
    matches["competitionCoefficient"] +
    matches["attackCoefficientAway"] +
    matches["defenseCoefficientHome"]
)

# show matches including predictions
matches.head()